<a href="https://colab.research.google.com/github/amitca71/colab/blob/main/pdfreader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
from pypdf import PdfReader
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
import os
import re
import pandas as pd
pdf_folder_path = f'{root_dir}/charles_data/'
os.listdir(pdf_folder_path)

In [37]:
#rfi_date has two values for the sake of the example. actually, the 2nd one covers the 1st one and can be left alone
#initially, only the 1st one was there and rfi_date was not extracted from RFI 0800 - Louver Support Framing at Loading Dock Soffit (RESPONSE without attachment).pdf
#see how it was debugged below
regex_str_dct={
    "rfi_title": [r"(?P<rfi_title>(.|\n)*)REQUEST FOR INFORMATION" ],
    "rfi_number": [r"(.|\n)*REQUEST FOR INFORMATION (?P<rfi_number>\d+)"],
    "rfi_date": [r"(.|\n)*DATE : (?P<rfi_date>\d+/\d+/\d+)", r"(.|\n)*DATE\s*:\s*(?P<rfi_date>\d+/\d+/\d+)"],
    "project_name": [r"(.|\n)*PROJECT:\n(?P<project_name>.*)"],
    "due_date": [r"(.|\n)*Response Required By:  \n(?P<due_date>\d+/\d+/\d+)"],
    "assigned_to": [r"(.|\n)*Fax: (\d+|\(|\)|\s|-)*\n(?P<assigned_to>.*)NUMBER"],
    "question": [r"(.|\n)*RELATED DOCUMENTS: Rev. Rev. Date\n.*\n(?P<question>(.|\n)*)QUESTION: .* Authored By:\nANSWER:" ,r"(.|\n)*QUESTION: .* Authored By:\n(?P<question>(.|\n)*)ANSWER:"],
    "answer": [r"(.|\n)*ANSWER:\n(?P<answer>(.|\n)*)\nDIRECTION TO SUB/VENDOR:"]
}

# "rfi_date": [r"(.|\n)*DATE : (?P<rfi_date>\d+/\d+/\d+)", r"(.|\n)*DATE\s*:\s*(?P<rfi_date>\d+/\d+/\d+)"],

In [ ]:
def get_value_by_regex(page, regex_str):
  return (re.match(regex_str, page))

In [38]:
rfis_meta_list=[]
for fl in os.listdir(pdf_folder_path):
  rfi_meta_dict={}
  pdf_file_path=f'{pdf_folder_path}/{fl}'
  pdf_reader=PdfReader(pdf_file_path)
  first_page_str=pdf_reader.pages[0].extract_text()
  for r in regex_str_dct.keys():
    for i in range (0, (len(regex_str_dct[r]))):
      matches=get_value_by_regex(first_page_str, regex_str_dct[r][i])
      if (matches):
#       print(matches.group(r))
        rfi_meta_dict[r]=matches.group(r)
        break;
#  print(rfi_meta_dict)
  if (len(rfi_meta_dict.keys())<len(regex_str_dct.keys())):
    print (f"file {pdf_file_path} does not contain all attributes")
    print("Missing values:", (set(regex_str_dct.keys()).difference(rfi_meta_dict.keys())))
  rfi_meta_pd=pd.DataFrame.from_dict(rfi_meta_dict,orient='index').T
  rfis_meta_list.append(rfi_meta_pd)

In [ ]:
print(first_page_str)

In [ ]:
full_pd=pd.concat(rfis_meta_list)
full_pd

In [40]:
full_pd.to_csv('/tmp/all_data.csv')

In [ ]:
!cat /tmp/all_data.csv

In [ ]:
rfi_meta_dict={}
pdf_file_path="/content/gdrive/My Drive//charles_data//RFI 0800 - Louver Support Framing at Loading Dock Soffit (RESPONSE without attachment).pdf"
pdf_reader=PdfReader(pdf_file_path)
first_page_str=pdf_reader.pages[0].extract_text()
print(first_page_str)
for r in regex_str_dct.keys():
  for i in range (0, (len(regex_str_dct[r]))):
    matches=get_value_by_regex(first_page_str, regex_str_dct[r][i])
    if (matches):
      print(matches.group(r))
      rfi_meta_dict[r]=matches.group(r)
      break;
print(rfi_meta_dict)

In [ ]:
first_page_str